# Elastic Net Regression on the Diabetes Dataset

This notebook walks through training an Elastic Net model using scikit-learn. Elastic Net blends **L1 (Lasso)** and **L2 (Ridge)** regularization:
- `alpha` controls the overall strength of regularization.
- `l1_ratio` controls the mix: `0` = Ridge-like, `1` = Lasso-like, in-between = Elastic Net.

Why use it?
- Handles **multicollinearity** better than Lasso alone.
- Can still perform **feature selection** unlike Ridge.
- Often outperforms Ridge/Lasso when features are correlated.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Load the dataset
The diabetes dataset has 10 numeric features and a continuous target (disease progression).

In [ ]:
dataset = load_diabetes(as_frame=True)
X = dataset.data
y = dataset.target
X.head(), y.head()

## 2. Train/test split and scaling
Elastic Net is sensitive to feature scales, so we standardize features inside a pipeline.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("elastic", ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=5000, random_state=42)),
])

pipeline

## 3. Train and evaluate
We'll report MSE, MAE, RMSE, and R².

In [ ]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print({"MSE": mse, "MAE": mae, "RMSE": rmse, "R2": r2})

## 4. Visualize predictions
A quick scatter plot of true vs. predicted values.

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.7, edgecolors="k")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--")
plt.xlabel("True values")
plt.ylabel("Predicted values")
plt.title("Elastic Net Regression: True vs Predicted")
plt.show()

## 5. Coefficient analysis
Higher absolute values mean stronger influence. With `l1_ratio=0.5`, some coefficients may be small but not exactly zero. Increasing `l1_ratio` pushes more coefficients toward zero; decreasing it makes the model more Ridge-like.

In [ ]:
coefs = pipeline.named_steps["elastic"].coef_
feature_names = X.columns

sorted_indices = np.argsort(np.abs(coefs))[::-1]

pd.DataFrame({
    "feature": feature_names[sorted_indices],
    "coefficient": coefs[sorted_indices]
}).head(10)